# OsmGt example

## Imports and prepare input parameters

In [4]:
from IPython.display import display

from bokeh.plotting import output_notebook
from bokeh.plotting import show

import geopandas as gpd

from shapely.geometry import Point
from shapely.ops import linemerge

from gdf2bokeh import Gdf2Bokeh

from osmgt import OsmGt

from graph_tool.topology import shortest_path


output_notebook()


location = "Roanne"


Loading BokehJS ...

## Get POIs

In [5]:
%%time

pois_gdf = OsmGt.pois_from_location(location).get_gdf()
pois_gdf_veto = pois_gdf.loc[pois_gdf["amenity"] == "restaurant"]
display(pois_gdf_veto)
pois_gdf["amenity"].unique()

2020-11-20 10:56:40 - OsmGtPoi        - INFO     : From location: Roanne
2020-11-20 10:56:40 - OsmGtPoi        - INFO     : Loading data...
2020-11-20 10:56:40 - OsmGtPoi        - INFO     : NominatimApi: Query 200:OK in 0.12 sec.
2020-11-20 10:56:41 - OsmGtPoi        - INFO     : OverpassApi: Query 200:OK in 0.77 sec.
2020-11-20 10:56:41 - OsmGtPoi        - INFO     : Formating data
2020-11-20 10:56:41 - OsmGtPoi        - INFO     : Prepare GeoDataframe
2020-11-20 10:56:41 - OsmGtPoi        - INFO     : GeoDataframe Ready


,addr:postcode,amenity,atm,change_machine,name,operator,phone,ref:FR:LaPoste,source,stamping_machine,...,type:FR:FINESS,service:bicycle:Bicycle_Sales_and_Service,service:bicycle:repair,check_date:opening_hours,contact:email,contact:facebook,source:opening_hours,clothes,waste,geometry
19,NaN,restaurant,NaN,NaN,L'avant première,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.06466 46.03677)
30,NaN,restaurant,NaN,NaN,Le Don Camillo,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.06670 46.04303)
121,NaN,restaurant,NaN,NaN,Courtepaille,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.08788 46.04880)
124,42300,restaurant,NaN,NaN,Maxi Kebab,NaN,+33 4 77 71 39 86,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.05962 46.03832)
126,42300,restaurant,NaN,NaN,Ô 28,NaN,+33 6 59 76 06 25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.06442 46.03830)
129,42300,restaurant,NaN,NaN,Taste & Wok,NaN,+33 4 77 78 05 06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.08688 46.05869)
142,42300,restaurant,NaN,NaN,Cira Restauration,NaN,+33 4 77 72 97 22,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.08293 46.06807)
144,42300,restaurant,NaN,NaN,Chez Doudou,NaN,+33477786966,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.07061 46.02983)
155,NaN,restaurant,NaN,NaN,Le Break,NaN,+33 6 20 21 11 81,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.07438 46.04584)
173,42300,restaurant,NaN,NaN,NOW WHY,NaN,+33477713096,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.06682 46.03685)


CPU times: user 154 ms, sys: 20.2 ms, total: 174 ms
Wall time: 1.06 s


array(['post_office', 'place_of_worship', 'police', 'theatre', nan,
       'toilets', 'drinking_water', 'recycling', 'waste_basket',
       'boat_rental', 'bicycle_rental', 'restaurant', 'car_wash',
       'cinema', 'school', 'fuel', 'fire_station', 'social_facility',
       'townhall', 'dancing_school', 'post_box', 'bar', 'pharmacy',
       'courthouse', 'community_centre', 'fast_food', 'college', 'bank',
       'nightclub', 'veterinary', 'library', 'driving_school',
       'charging_station', 'bicycle_parking'], dtype=object)

### Isochrone from distances

In [6]:
%%time

trip_speed = 3  # km/h

isochrones_polygon_values = {
    20: "#d9ef8b",
    50: "#fee08b",
    75: "#f46d43",
    100: "#8B4513"
}
# 250 = 250 meters ; 500 = 500 meters ; etc

isochrones_lines_values = {
    20: "#668100",
    50: "#e2a803",
    75: "#962603",
    100: "#000000",
    9999: "grey"
}


output = OsmGt.isochrones_from_multiple_locations_on_area(
    pois_gdf_veto,
    location,
    list(isochrones_polygon_values.keys()),  # meters
    trip_speed,
    mode="pedestrian"
)


# isochrones_lines["color"] = isochrones_lines["iso_distance"].map(lambda x: isochrones_lines_values[x])


# layers_to_add = [
#     {
#         "input_gdf": isochrones_lines,
#         "legend": "iso_distance",
#         "color": "color",
#         "line_width": 2
#     }
# ]

# my_shortest_path_map = Gdf2Bokeh(
#     "Isochrones from distance - from OsmGT (https://github.com/amauryval)",
#     layers=layers_to_add
# )
# show(my_shortest_path_map.figure)


2020-11-20 10:56:41 - OsmGtIsochrone  - INFO     : Isochrone processing...
2020-11-20 10:56:41 - OsmGtIsochrone  - INFO     : From location: Roanne
2020-11-20 10:56:41 - OsmGtIsochrone  - INFO     : Loading data...
2020-11-20 10:56:41 - OsmGtIsochrone  - INFO     : NominatimApi: Query 200:OK in 0.12 sec.
2020-11-20 10:56:44 - OsmGtIsochrone  - INFO     : OverpassApi: Query 200:OK in 2.02 sec.
2020-11-20 10:56:44 - OsmGtIsochrone  - INFO     : Rebuild network data
2020-11-20 10:56:44 - OsmGtIsochrone  - INFO     : Network cleaning...
2020-11-20 10:56:44 - OsmGtIsochrone  - INFO     : Starting: Adding new nodes on the network
2020-11-20 10:56:44 - OsmGtIsochrone  - INFO     : Find nearest line for each node
2020-11-20 10:56:44 - OsmGtIsochrone  - INFO     : Split line
2020-11-20 10:56:44 - OsmGtIsochrone  - INFO     : Topology lines checker: to add: 10, to split: 10
2020-11-20 10:56:44 - OsmGtIsochrone  - INFO     : Starting: Find intersections
2020-11-20 10:56:44 - OsmGtIsochrone  - INF

EmptyData: GeoDataframe creation is impossible, because no data has been found

In [ ]:
output.to_file("package.gpkg", layer=f'aaaaa', driver="GPKG")

In [ ]:
isochrones_lines_values = {
    20: "#668100",
    50: "#e2a803",
    75: "#962603",
    100: "#000000",
    9999: "grey"
}

output["color"] = output["iso_distance"].map(lambda x: isochrones_lines_values[x])


layers_to_add = [
    {
        "input_gdf": output,
        "legend": "iso_distance",
        "color": "color",
        "line_width": 2
    },
    {
        "input_gdf": pois_gdf_veto,
        "legend": "Source node",
        "style": "circle",
        "color": "red",
        "size": 10
    },
]

layers_map = Gdf2Bokeh(
    "test",
    layers=layers_to_add
)
show(layers_map.figure)